<a href="https://colab.research.google.com/github/AjayBora002/Weather-Prediction/blob/main/weather_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Section 1 : Import Libraries**

In [1]:
import requests # This library helps us to fetch data from API
import pandas as pd # FOr handling and analyzing data
import numpy as np # for numerical operations
from sklearn.model_selection import train_test_split # to split data into training and testing sets
from sklearn.preprocessing import LabelEncoder # to convert catogerial data into numerical values
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor #model for classification and regression tasks
from sklearn.metrics import mean_squared_error # to measure the accuracy of our prediction
from datetime import datetime, timedelta # to handle date and time
import pytz

In [2]:
API_KEY ='2d735c6f248acf337e16b7d15c5837ac' #REPLACE WITH YOUR ACTUAL API key
BASE_URL='https://api.openweathermap.org/data/2.5/' #base url for making API requests


**1.Fetch Current Weather Data**

In [3]:
def get_current_weather(city):
  url = f"{BASE_URL}weather?q={city}&appid={API_KEY}&units=metric" #construct the API request URL
  response = requests.get(url) # sends the get request to API
  data = response.json()
  return {
      'city' : data['name'],
      'current_temp' : round(data['main']['temp']),
      'feels_like' : round(data['main']['feels_like']),
      'temp_min' : round(data['main']['temp_min']),
      'temp_max' : round(data['main']['temp_max']),
      'humidity' : round(data['main']['humidity']),
      'description':data['weather'][0]['description'],
      'country' :data['sys']['country'],
      'Wind_Gust_Speed' : data['wind']['speed'],
      'Wind_Gust_Dir' : data['wind']['deg']

  }

2**.Read Historical Data**

In [4]:
def read_historical_data(filename):
  df = pd.read_csv("https://raw.githubusercontent.com/AjayBora002/Weather-Prediction/refs/heads/main/weather.csv") #load csv file into dataframe
  df=df.dropna() #remove rows with missing values
  df=df.drop_duplicates()
  return df

**3. Prepare data for training**

In [5]:
def prepare_data(data):
  le = LabelEncoder() #create a labelencodr instance
  data['RainTomorrow'] = le.fit_transform(data['RainTomorrow'])

  #define the feature variable and target variable
  X=data[['MaxTemp','MinTemp','Temp','Humidity','WindGustSpeed','WindGustDir']]
  y = data['RainTomorrow'] #target variable
  return X,y,le

**4. train rain prediction model**

In [6]:
def train_rain_model(X,y):
  X_train, X_test, y_train ,y_test = train_test_split(X,y, test_size=0.2, random_state=42)
  model = RandomForestClassifier(n_estimators=100, random_state=42)
  model.fit(X_train, y_train) #train the model

  y_pred = model.predict(X_test)  #to make predictions on test set
  print("mean squared error for rain model")
  print(mean_squared_error(y_test, y_pred))
  return model

**5.prepare regression data**

In [7]:
def prepare_regression_data(data, feature):
  X,y = [], []

  for i in range(len(data) -1):
    X.append(data[feature].iloc[i])
    y.append(data[feature].iloc[i+1])
  X=np.array(X).reshape(-1, 1)
  y=np.array(y)
  return X,y

6**.train regression model**

In [8]:
def train_regression_model(X,y):
  model = RandomForestRegressor(n_estimators=100, random_state=42)
  model.fit(X, y)
  return model


**7. predict future**

In [9]:
def predict_future(model, current_value):
  predictions = [current_value]

  for i in range(5):
    next_value = model.predict(np.array([[predictions[-1]]]))
    predictions.append(next_value[0])
  return predictions[1:]


**8. Weather analysis functi**on

In [10]:
def weather_view():
  city = input('Enter any city name: ')
  current_weather = get_current_weather(city)

  #load historical data
  historical_data= read_historical_data('/content/weather.csv')

  #prepare and train the rain prediction model

  X, y, le = prepare_data(historical_data)

  rain_model = train_rain_model(X,y)

  current_data = {
      'MaxTemp': current_weather['temp_max'],
      'MinTemp': current_weather['temp_min'],
      'Temp': current_weather['current_temp'],
      'Humidity': current_weather['humidity'],
      'WindGustSpeed': current_weather['Wind_Gust_Speed'],
      'WindGustDir': current_weather['Wind_Gust_Dir'],

   }


  current_df = pd.DataFrame([current_data])

   #rain prediction

  rain_prediction = rain_model.predict(current_df)[0]

   #prepare regression model for temp and humidity

  X_temp, y_temp = prepare_regression_data(historical_data, 'Temp')
  X_hum, y_hum = prepare_regression_data(historical_data, 'Humidity')
  temp_model = train_regression_model(X_temp, y_temp)
  hum_model = train_regression_model(X_hum, y_hum)

#predict future temp and hum

  future_temp = predict_future(temp_model, current_weather['temp_min'])

  future_humidity = predict_future(hum_model, current_weather['humidity'])

  timezone = pytz.timezone('Asia/Karachi')
  now = datetime.now(timezone)
  next_hour = now + timedelta(hours=1)
  next_hour = next_hour.replace(minute=0, second=0, microsecond=0)
  future_times = [(next_hour + timedelta(hours=i)).strftime("%H:00") for i in range(5)]

  #display results

  print(f"City : {city},{current_weather['country']}")
  print(f"Current Temperature: {current_weather['current_temp']}")
  print(f"Feels Like : {current_weather['feels_like']}")
  print(f"Minimum Temperature : {current_weather['temp_min']}°C")
  print(f"Maximum Temperature : {current_weather['temp_max']}°C")
  print(f"Humidity : {current_weather['humidity']}%")
  print(f"Weather Prediction : {current_weather['description']}")
  print(f"Rain Prediction : {'Yes' if rain_prediction else 'No'}")

  print("\nFuture Temperature Predictions : ")

  for time, temp in zip(future_times, future_temp):
    print(f"{time}: {round(temp, 1)}°C")

  print("\nFuture Humidity Predictions: ")

  for time, humidity in zip(future_times, future_humidity):
    print(f"{time}: {round(humidity,1)}%")

weather_view()









Enter any city name: haldwani
mean squared error for rain model
0.3333333333333333
City : haldwani,IN
Current Temperature: 37
Feels Like : 35
Minimum Temperature : 37°C
Maximum Temperature : 37°C
Humidity : 19%
Weather Prediction : broken clouds
Rain Prediction : No

Future Temperature Predictions : 
17:00: 24.8°C
18:00: 25.3°C
19:00: 26.4°C
20:00: 26.8°C
21:00: 26.8°C

Future Humidity Predictions: 
17:00: 14.4%
18:00: 11.6%
19:00: 13.6%
20:00: 11.7%
21:00: 13.6%


ValueError: Mountpoint must be in a directory that exists